In [ ]:
# Reference used for update formulas of Multivariate GMM: https://www.python-course.eu/expectation_maximization_and_gaussian_mixture_models.php

In [7]:
import numpy as np
from scipy.stats import multivariate_normal
from copy import deepcopy

In [8]:
#kmeans++

In [9]:
# data = []
def kmeanspp(data):
    k = 2
    n = len(data)
    random_centre = np.random.choice(list(range(n)),size = 1)[0]
    centre = []
    centre.append(data[random_centre])
    centre_dict = dict()
    nearest_centre = [random_centre for i in range(n)]
    centre_dict[random_centre] = list(range(n))
    for i in range(k-1):
        p = [np.linalg.norm(data[nearest_centre[i]]-data[i])**2 for i in range(n)]
        p = p/sum(p)
        new_cent = np.random.choice(list(range(n)), p = p, size = 1)[0]
        centre.append(new_cent)
        centre_dict[new_cent] = []
        for point in range(n):
            if np.linalg.norm(data[point] - data[new_cent]) < np.linalg.norm(data[point] - data[nearest_centre[point]]):
                centre_dict[nearest_centre[point]].remove(point)
                centre_dict[new_cent].append(point)
                nearest_centre[point] = new_cent
    cost = 0
    for cent in centre_dict:
        cost+=sum([np.linalg.norm(data[cent]-data[pt])**2 for pt in centre_dict[cent]])
    # print("kmeans cost after initialization", cost)
    n_iter = 5
    index_dict = dict() 
    for iter_no in range(n_iter):
        new_centers = []
        for cent in centre_dict:
            new_cent = np.mean([data[pt] for pt in centre_dict[cent]], axis = 0)
            new_centers.append(new_cent)
            index_dict[tuple(new_cent)] = cent 
            centre_dict[cent] = []
        # print(len(new_centers[0]))
        # print([np.linalg.norm(data[1] - cent) for cent in new_centers])
        cost = 0
        for point in range(n):
            nearest_centre_full = tuple(new_centers[np.argmin([np.linalg.norm(data[point] - cent) for cent in new_centers])])
            cent = index_dict[nearest_centre_full]
            # print(cent)
            
            centre_dict[cent].append(point)
            cost+=np.linalg.norm(nearest_centre_full-data[point])**2
        # print(cost, ":cost after ", iter_no+1, "-iteration")
    return np.array(list(index_dict.keys()))
    # return centre


In [10]:
#EM GMM fit

In [34]:
from sklearn.mixture import GaussianMixture
def GMM(data):
    prior = [0.5, 0.5]
    #Random centers
    n = len(data)
    mu1 = data[np.random.choice(list(range(n)),size = 1)[0]]
    mu2 = data[np.random.choice(list(range(n)),size = 1)[0]]
    mu = np.stack((mu1, mu2))
    # print(mu.shape)
    # mu = np.random.randint(min(data[:,0]),max(data[:,0]),size=(2,len(data[0])))
    cov = np.zeros((2, 32, 32))
    # To avoid calculation of inverse error due to not having full rank issues.
    reg_cov = 1e-6*np.identity(32)
    for i in range(len(cov)):
        np.fill_diagonal(cov[i], 5)
    n_iter = 100

    for i in range(n_iter):
        r_ic = np.zeros((10000,len(cov)))
        sum_=0
        for pi_c,mu_c,cov_c in zip(prior,mu,cov):
            cov_c += reg_cov
            sum_ += pi_c*multivariate_normal(mean=mu_c,cov=cov_c).pdf(data) 
        for mean, covariance, p, r in zip(mu, cov, prior, range(len(r_ic[0]))):
            covariance+=reg_cov
            mn = multivariate_normal(mean=mean,cov=covariance)
            r_ic[:, r] = p*mn.pdf(data)/sum_
        mu = []
        cov = []
        prior = []
        log_liklihood = []
        for c in range(len(r_ic[0])):
            m_c = np.sum(r_ic[:, c], axis = 0)
            # print(m_c)
            mu_c = (1/m_c)*np.sum(data*r_ic[:,c].reshape(len(data), 1), axis = 0)
            mu.append(mu_c)
            # print(data.shape)
            cov.append(np.cov(data.T, 
                    aweights=(r_ic[:,c]/m_c), 
                    bias=True))
            prior.append(m_c/np.sum(r_ic))
        return np.array(mu)
    # gmm = GaussianMixture(n_components=2, covariance_type='spherical', max_iter=50)
    # gmm.fit(data)
    # # print(gmm.means_)
    # return gmm.means_

In [35]:
result = []
for c in [0.5, 1, 1.5, 2, 3, 4, 8]:
    mean1 = [0]*32
    mean2 = [c*np.random.normal(0.0, 1, 1)[0]]*32
    std = np.identity(32)
    # data = np.random
    data1 = np.random.multivariate_normal(mean1, std, (5000,))
    data2 = np.random.multivariate_normal(mean2, std, (5000,))
    data = np.concatenate((data2, data1))
    # print(data.shape)
    kmean_cent = kmeanspp(data)
    print("For c = ", c)
    print("kmeans done")
    a = np.linalg.norm(kmean_cent[0]-mean1)+np.linalg.norm(kmean_cent[1]-mean2)
    b = np.linalg.norm(kmean_cent[1]-mean1)+np.linalg.norm(kmean_cent[0]-mean2)
    print("Kmeans cent dist", min(a,b))
    gmm_cent = GMM(data)
    print("GMM done")
    x = np.linalg.norm(gmm_cent[0]-mean1)+np.linalg.norm(gmm_cent[1]-mean2)
    y = np.linalg.norm(gmm_cent[1]-mean1)+np.linalg.norm(gmm_cent[0]-mean2)
    # print(min(a,b))
    result.append([c, min(a,b), min(x,y)])
    print("GMM cent dist", min(x,y))


For c =  0.5
kmeans done
Kmeans cent dist 0.687999831597058
GMM done
GMM cent dist 1.3016526412740161
For c =  1
kmeans done
Kmeans cent dist 1.9788134061399765
GMM done
GMM cent dist 2.1711537427948335
For c =  1.5
kmeans done
Kmeans cent dist 0.19282551838725923
GMM done
GMM cent dist 6.189097669046462
For c =  2
kmeans done
Kmeans cent dist 1.4467139706834216
GMM done
GMM cent dist 2.6734939363549204
For c =  3
kmeans done
Kmeans cent dist 2.353915769777936
GMM done
GMM cent dist 1.8092019983826884
For c =  4
kmeans done
Kmeans cent dist 0.16340439085922517
GMM done
GMM cent dist 0.16854735229389617
For c =  8
kmeans done
Kmeans cent dist 0.18149552556022558
GMM done
GMM cent dist 20.21364548925851


In [36]:
from tabulate import tabulate
x = tabulate(np.array(result),tablefmt="latex", floatfmt=".2f")
print(x)

\begin{tabular}{rrr}
\hline
 0.50 & 0.69 &  1.30 \\
 1.00 & 1.98 &  2.17 \\
 1.50 & 0.19 &  6.19 \\
 2.00 & 1.45 &  2.67 \\
 3.00 & 2.35 &  1.81 \\
 4.00 & 0.16 &  0.17 \\
 8.00 & 0.18 & 20.21 \\
\hline
\end{tabular}
